3. 전처리를 통한 모델 데이터셋 생성

In [ ]:
#1. 패키지 임포트

import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#2. 데이터 설명
'''
'''

In [ ]:
#3. 데이터 불러오기(csv)

PATH = '../Data/'

#sales_df = pd.read_csv(PATH+'sales.csv) # 2019년도 데이터만 추출
sales_df = pd.read_csv(PATH+'sales_2019.csv')
store_df = pd.read_csv(PATH+'store_cities.csv')
product_df = pd.read_csv(PATH+'product_hierarchy.csv')

In [ ]:
# 판매 데이터 확인
sales_df.head()

# 매장 데이터 확인
sales_df.describe()

# 변수별 unique 데이터 확인
sales_df.unique()

In [ ]:
# 4. 필요한 데이터 유지(필요없는 변수 삭제)

### drop함수 사용하여 'year'변수 삭제
sales_df.drop('year', axis=1, inplace=True)

### 필요 변수만 유지 - 'year' 변수만 제외하고 유지
columns = ['product_id', 'store_id', 'date','sales', 'revenue', 'stock', 'price', 'promo_type_1', 'promo_bin_1', 
           'promo_type_2', 'promo_bin_2', 'promo_discount_2', 'promo_discount_type_2']
sales_df = sales_df[columns]

In [ ]:
# 5. 결측치 변수 삭제
# 결측치 여부 - 각 변수별 결측치 개수 확인

sales_df.isna().sum()

sales_df.shape

In [ ]:
# 'sales' 변수 기준을 결측치 행 삭제
# 방법 1 - 결측치를 제외한 인덱스 유지
sales_df = sales_df[sales_df['sales'].insa()!=True].reset_index(drop=True)
# 방법 2 - 결측치 인덱스를 드롭한 남은 인덱스 추출
sales_df = sales_df.iloc[sales_df['sales'].dropna().index].reset_index(drop=True)


sales_df.shape

In [ ]:
# 결측치 제거 후 다시확인
sales_df.isna().sum()

# 가격 정보가 있는 데이터
sales_df[sales_df.price>0].head()

# 가격정보가 없는 데이터
sales_df[sales_df.price.isna()].head()

sales_df.promo_type_1.unique()

sales_df.shape


In [ ]:
# 5. 결측치 채우기(문자열 변수의 결측치는 "으로 변경)

# promo_bin_1 데이터 값 확인
sales_df.promo_bin_1.unique()

# 결측치 수정
sales_df.promo_bin_1.fillna('', inplace=True)

# 수정 후 데이터 값 재확인
sales_df.promo_bin_1.unique()

# promo_type_2 값 확인
sales_df.promo_type_2.unique()

# promo_bin_2 값 확인
sales_df.promo_bin_2.unique()

# 결측치 수정
sales_df.promo_bin_2.fillna('', inplace=True)

sales_df.promo_bin_2.unique()

sales_df.promo_discount_type_2.unique()

sales_df.promo_discount_type_2.fillna('', inplace=True)

sales_df.promo_discount_type_2.unique()


In [ ]:
# 6.가격 결측치 채우기

# 가격 결측 데이터 확인
sales_df[sales_df.price.isna()].head()

# 가격 데이터와 결측치 데이터 분리
# 가격정보 유 데이터 df1으로 생성
df1 = sales_df[sales_df.price.isna()! = True]
# 가격정보 무 데이터 df0으로 생성
df0 = sales_df[sales_df.price.isna() == True]

# 가격정보 없는 매장의 상품 정보가 가격 정보가 있는 데이터에서 존재하는지 확인
# 상품코드/매장코드 기준으로 병합
# 날짜별 가격 정보 포함
# df1 의 상품코드/매장코드/가격/날짜 와 df0의 상품코드/ 매장코드로 상품코드/매장코드 기준으로 병합 후 dfz로 생성
dfz = pd.merge(df1[['product_id', 'store_id', 'price','date']], df0[['product_id', 'store_id']], on=['priduct_id', 'store_id'], how='inner')

# 상품코드/매장코드/날짜/판매가격 순으로 정렬 (날짜와 판매가격은 낮은 순으로)
dfz = dfz.sort_values(['product_id', 'store_id', 'date', 'price'], ascending=[True, True, True, True]).reset_index(drop=True)

# 가격정보 없는 데이터에 가격 정보 병합
# 가격정보 없는 데이터에서 가격 변수 삭제 후 병합
df0.drop('price', axis=1, inplace=True)
df0 = pd.merge(df0, dfz, on=['product_id', 'store_id'], how = 'left')

# df0 데이터 확인
df0.shape

# 수정 후 가격확인 건 df01으로 생성
df01 = df0[df0.price.isna()!=True]
df01.shape

# 수정 후 가격 미확인 건 df00으로 생성
df00 = df0[df0.price.isna()]
df00.shape

# 수정 후에도 확인 안되는 상품은 매장구분없이 동일 상품 가격 정보 추출
dfz0 = dfz[dfz.product_id.isin(df00.product_id)]

# 상품의 최소가격으로 맵핑하기 위해 최소가격 기준으로 그룹핑
dfz0 = dfz0.groupby('product_id').min()['price'].reset_index()

# dfz0로 생성한 최고가격 정보를 df00 테이블에 업데이트
# df00의 가격 변수를 삭제가 필요함(dfz0와 df00에 각 가격 변수가 있기 때문에, 동일명변수가 존재시, _x 와 _y 로 각각 생성됨)
df00.drop('price', axis=1, inplace=True)
df00 = pd.merge(df00, dfz0, on='product_id', how='left')

# df1, df01, df00으로 가격정보를 다 업데이트 한 테이블을 sales_df_fixed로 병합
sales_df_fixed = pd.concat([df1, df01, df00]).reset_index(drop=True)


In [ ]:
# 7. One-hot encoding

# 가격업데이트 이후에도 가겨 정보가 없는 제품의 경우 원핫이노딩으로 가격유무 여부 변수로 구분자 생성
# 가격 결측치 여부
sales_df_fixed.loc[sales_df_fixed['price'].isna()==True, 'price_ohe'] = 1
sales_df_fixed.loc[sales_df_fixed['price'].isna()==False, 'price_ohe'] = 0
sales_df_fixed['price_ohe'] = sales_df_fixed['price_ohe'].astype(int)

# promo_type_1 유형별 원핫인코딩
for i in sales_df_fixed.promo_type_1.unique().tolist():
    if i == '':
        continue
    else:
        # 변수 생성
        sales_df_fixed['promo_type_1_'+i] = 0
        sales_df_fixed.loc[sales_df_fixed.promo_type_1==i, 'promo_type_1_'+1] = 1
        
# promo_type_1 변수 삭제
sales_df_fixed.drop('promo_type_1', axis=1, inplace=True)

# 각 변수의 유형별 원핫인코딩
columns = ['promo_bin_1', 'promo_type_2', 'promo_bin_2', 'promo_discount_type_2']
for col in columns:
    for i in sales_df_fixed[col].unique().tolist():
        if i =='':
            continue
        else:
            sales_df_fixed[col+'_'+i] = 0
            sales_df_fixed.loc[sales_df_fixed[col]==i, col+'_'+i] = 1
            
# 원핫인코딩 완료된 변수 삭제
sales_df_fixed.drop(columns, axis=1, inplace=True)

# 할인율 없는 정보는 0으로 대체
sales_df_fixed.promo_discount_2.fillna(0, inplace=True)

# 가격 없는 정보는 0으로 대체
sales_df_fixed.price.fillna(0, inplace=True)


In [ ]:
# 8. 그룹핑

# product_id 기준으로 판매수량 및 이익액 총합
product_grouping =sales_df_fixed[['product_id', 'sales', 'revenue']].groupby('product_id').sum().reset_index()

# store_id 기준으로 판매수량 및 이익액 총합
store_grouping = sales_df_fixed[['store_id', 'sales', 'revenue']].groupby('store_id').sum().reset_index()

# product_id + store_id 기준으로 판매수량 및 이익액 총합
product_store_grouping = sales_df_fixed[['product_id', 'store_id', 'sales', 'revenue']].groupby('product_id', 'store_id').sum().reset_index()

# 매장별 상품별 판매수량 pivot table
store_product_pivot = sales_df_fixed.pivot_table(index='product_id', columns='store_id', values='sales').fillna(0)


In [ ]:
# 9. 데이터 그룹핑 및 합/평균/최소/최대 값 구하기

# 상품의 최저값 구하기 - 0 값 제외
sales_df_fixed0 = sales_df_fixed[sales_df_fixed.price_ohe==0]
min_price = sales_df_fixed0[['product_id', 'price']].groupby('product_id').min().reset_index()

# 상품의 최고값 구하기
max_price = sales_df_fixed0[['product_id', 'price']].groupby('product_id').max().reset_index()

# 최저/최고값 비교하기
price_compare = pd.merge(min_price, max_price, on='product_id')   # on= : merge의 기준이 되는 Key 변수

# 변수명 변경
price_compare.columns = ['product_id', 'min_price', 'max_price']

# 최고/최저값 차이
price_compare['price_diff'] = price_compare['max_price'] - price_compare['min_price']

# 최고-최저 동일 상품
same_price = price_compare[price_compare.price_diff==0]

# 최고!=최저 다른 상품
diff_price = price_compare[price_compare.price_diff!=0]


In [ ]:
# 10. 테이블 병합 Merging & Concatenating left, inner, outer

df = pd.merge(sales_df_fixed, sales_df, on = 'store_id', how='left')

# storetype_id, city_id 유형별 원핫인코딩
columns = ['storetype_id', 'city_id']
for col in columns:
    for i in df[col].unique().tolist():
        # 변수 생성
        df[col +'_'+i] = 0
        df.loc[df[col]==i, col+'_'+i] = 1
        
# promo_type_1 변수 삭제
df.drop(columns, axis=1, inplace=True)

# 매장 면적의 분포를 확인 후 대중소 기준으로 구분함
plt.hist(df['store_size'], bins=20)

In [ ]:
# 11. 함수활용

# 아래 매장 면적 기준으로 3개 그룸으로 구분
# 0~30
# 30~50
# 50<

def storesize_grouping(x):
    if x<=30:
        return 1
    elif x<=50:
        return 2
    else:
        return 3
    
df['storesize_grouping'] = df['store_size'].apply(storesize_grouping)


# 구분한 매장을 store_size 원핫인코딩으로 변환
df.loc[df['storesize_grouping']==1, 'storesize_samll'] = 1
df.loc[df['storesize_grouping']==2, 'storesize_medium'] = 1
df.loc[df['storesize_grouping']==3, 'storesize_large'] = 1


# NA 값 0 변경
df[['storesize_small', 'storesize_medium', 'storesize_large']] = df[['storesize_small', 
                                                                     'storesize_medium', 'storesize_large']].fillna(0)

# store_size 변수 삭제
df.drop(['store_size', 'storesize_grouping'], axis=1, inplace=True)

## 매장 사이즈별 원핫인코딩을 두단계 (분리 + 원핫인코딩)이 아닌 한단계로 실행 가능

df.head()

# 데이터 병합 또는 전처리 순서는 데이터 사이즈와 실행 시간을 고려하여 실행함
# df에 product_df를 병합 후 원핫인코딩 또는 다른 계산식을 하는 것과
# product_df에서 선 처리 후 df에 병합하는 방법에 실행 시간 및 연산량에 차이가 있음


# df 데이터 수
df.shape

# 전체 상품 정보데이터 수
product_df.shape

# 2019년에 판매된 상품의 정보는 628개 상품
product_df[product_df.product_id.isin(df.product_id.unique())].shape

# 97만건으로 계속 전처리 하는것 보다, 628 건으로 우선 전처리 후 병합하는 방법이 효율적임
product_df = product_df[product_df.product_id.isin(df.product_id.unique())].reset_index(drop=True)

# 상품별 가로, 세로, 높이로 부피 산출 함수를 생성
def get_volume(x, y, z):
    return x*y*z
product_df['product_volume'] = get_volume(product_df['product_length'], product_df['product_depth'], product_df['product_width'])

# 사이즈 정보가 누락으로 결측치 확인
product_df[product_df.product_volume.isna()]

# cluster_id 기준으로 제품 사이즈 정보가 있는 정보를 제품사이즈 정보가 없는 제품에 평균값으로 일괄적용
# 정보 대체 방식은 case by case 로 다양한 그러나 적정한 방법으로 적용해 볼 수 있음.

# 결측치 개수 확인
product_df.product_volume.isna().sum()

# cluster_0 기준의 평균값 대체
product_df.loc[(product_df['cluster_id']=='cluster_0')&(product_df['product_volume'].isna()), 'product_volume'] = product_df[
    product_df['cluster_id']=='cluster_0']['product_volume'].mean()

# 결측치 보완 후 결측치 잔여 여부 확인
product_df.product_volume.isna().sum()

# 상품 부피 사이즈는 백분위수 기준 10 단계로 그룹화
labels = np.arange(1, 11)
product_df['volume_qcut'] = pd.qcut(product_df['product_volume'], q = 10, labels=labels)

# 부피 qcut 유형별 원핫인코딩
for i in sorted(set(product_df.volume_qcut)):
    # 변수 생성
    product_df['volume_' + str(i)] = 0
    product_df.loc[product_df['volume_qcut']==i, 'volume_'+str(i)] = 1
    
# 상품 대분류 개수
product_df['hierarchy1_id'].nunique()

# 상품 중분류 개수
product_df['hierarchy2_id'].nunique()

# 대분류 중분류 기준으로 원핫인코딩
# hierarchy1_id, hierarchy2_id 유형별 원핫인코딩
columns = ['cluster_id', 'hierarchy1_id', 'hierarchy2_id']

for col in columns:
    for i in product_df[col].unique().tolist():
        # 변수생성
        product_df[i] = 0
        product_df.loc[product_df[col]==i, i] = 1
    
# 원핫인코딩 등 변수 생성이 완료된 원래 변수는 삭제
product_df.drop(['product_length', 'product_depth', 'product_width', 'product_volume', 'volume_qcut', 
                 'cluster_id', 'hierarchy1_id', 'hierarchy2_id', 'hierarchy3_id', 'hierarchy4_id', 'hierarchy5_id'], 
                axis=1, inplace=True)

product_df.head()

# 변수 전처리가 완료된 상품 데이터를 df 데이터로 병합
df = pd.merge(df, product_df, on='product_id', how='left')

df.shape

# 결측치 개수 확인
df.isna().sum().sum()


In [ ]:
# 12. 시계열 데이터 생성하기 Shifting 과 Rolling

# 날짜별 판매량 시계열 데이터 생성
ts = df[['date', 'sales']]

# 날짜별로 총 판매량 그룹핑
ts = ts.groupby('date').sum()

# 시각화
plt.plot(ts.sales)

ts.head()

ts.tail()

# 판매량 shifting
ts['sales_p1'] = ts.sales.shift(1)
ts['sales_m1'] = ts.sales.shift(-1)

# 판매량 5 단위로 Rolling
# 합
ts['sales_sum5'] = ts.sales.rolling(5).sum()
# 최소
ts['sales_min5'] = ts.sales.rolling(5).min()
# 최대
ts['sales_max5'] = ts.sales.rolling(5).max()
# 평균
ts['sales_avg5'] = ts.sales.rolling(5).mean()

ts


